<a href="https://colab.research.google.com/github/solita/a-ice/blob/main/notebooks/RandomForest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
path = '/content/drive/Shareddrives/A-ICE/Data/AIS+ICE/data.csv'
df = pd.read_csv(path)
df = df[df.DWT != 0]
df['PWRDWTRATIO'] = df['POWER']/df['DWT']
df.dropna(axis=0,inplace=True)

def create_random_column(length, mean, std):
    return list(np.random.normal(loc=mean,scale=std,size=length))

random_0_1 = create_random_column(len(df), 0, 1)
random_0_100 = create_random_column(len(df), 0, 100)
random_100_1 = create_random_column(len(df), 100, 1)
random_neg100_1 = create_random_column(len(df), -100, 1)
random_100_100 = create_random_column(len(df), 100, 100)
random_neg100_100 = create_random_column(len(df), -100, 100)

df['random1'] = random_0_1
df['random2'] = random_0_100
df['random3'] = random_100_1
df['random4'] = random_neg100_1
df['random5'] = random_100_100
df['random6'] = random_neg100_100



In [6]:

def mean_error(y_test, pred):
    return np.mean(np.abs(pred-y_test))
            
def error_percent(y_test, pred):
    percent = np.divide((y_test-pred), y_test).astype(float)
    return percent

IMOS = df.IMO.unique()
errors = []
importance = []
for imo in IMOS:
    train_set = df.loc[df.IMO==imo]
    test_set = df.loc[df.IMO!=imo]
    x_train = train_set[['POWER', 'DRAFT', 'LENGTH', 'BREADTH', 'SPEED',
       'GRT', 'NRT', 'DWT', 'PWRDWTRATIO', 'KEMI_WIND_DIR', 'KEMI_WIND_SPEED',
       'KEMI_TEMPERATURE_WEEK', 'KEMI_TEMPERATURE_MONTH', 'KALAJOKI_WIND_DIR',
       'KALAJOKI_WIND_SPEED', 'KALAJOKI_TEMPERATURE_WEEK',
       'KALAJOKI_TEMPERATURE_MONTH', 'ICEACT', 'ICETCK', 'ICEMAX', 'ICEMIN',
       'TYYPPI','random1','random2','random3','random4','random5','random6']]
    x_test = test_set[['POWER', 'DRAFT', 'LENGTH', 'BREADTH', 'SPEED',
       'GRT', 'NRT', 'DWT','PWRDWTRATIO', 'KEMI_WIND_DIR', 'KEMI_WIND_SPEED',
       'KEMI_TEMPERATURE_WEEK', 'KEMI_TEMPERATURE_MONTH', 'KALAJOKI_WIND_DIR',
       'KALAJOKI_WIND_SPEED', 'KALAJOKI_TEMPERATURE_WEEK',
       'KALAJOKI_TEMPERATURE_MONTH', 'ICEACT', 'ICETCK', 'ICEMAX', 'ICEMIN',
       'TYYPPI','random1','random2','random3','random4','random5','random6']]
    y_train = train_set['KNOTS_OVER_GROUND']
    y_test = test_set['KNOTS_OVER_GROUND']
    y_train = np.ravel(y_train)
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    #prediction = np.reshape(prediction, (len(prediction),1))
    errors.append([mean_error(y_test,prediction)])
    importance.append(model.feature_importances_)
    
plt.pcolormesh(importance)
plt.colorbar()

KeyboardInterrupt: ignored